In [ ]:
import urllib.request, time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
url = 'https://timesofindia.indiatimes.com/city/{0}/{1}?cfmid={2}'

In [ ]:
num_pages = 5
cities = ['mumbai', 'delhi', 'hyderabad', 'lucknow', 'gujarat']
full_news = []
ids = {'crime_news': 2000000, 'civic_issues': 14000000, 'politics': 11000000, 'school': 5000000}

In [ ]:
news = []
for city in cities:
  for key, value in ids.items():
    for page in range(2, num_pages + 1):
        print('processing page :', page)
        curr_url = url.format(city, page, value)
        print(curr_url)
        
        try:
            content = requests.get(curr_url)                        
        except Exception as e:                                
            continue                                       
        time.sleep(0.5)
        soup = BeautifulSoup(content.text,'html.parser')
        links = soup.find('div', attrs={'id': "c_articlelist_stories_2"})
        try:
          spans_content = links.find_all('span',  attrs={'class': "w_tle"})
        except:
          continue
        for link in spans_content:
          title = link.text
          links = link.find('a')['href']
          if (links[0] == '/'):
            links = 'https://timesofindia.indiatimes.com{}'.format(links)
          else:
            continue
          # print(title, links)
          news.append([title, links, city, key])
          # print(news)

processing page : 2
https://timesofindia.indiatimes.com/city/mumbai/2?cfmid=2000000
processing page : 3
https://timesofindia.indiatimes.com/city/mumbai/3?cfmid=2000000
processing page : 4
https://timesofindia.indiatimes.com/city/mumbai/4?cfmid=2000000
processing page : 5
https://timesofindia.indiatimes.com/city/mumbai/5?cfmid=2000000
processing page : 2
https://timesofindia.indiatimes.com/city/mumbai/2?cfmid=14000000
processing page : 3
https://timesofindia.indiatimes.com/city/mumbai/3?cfmid=14000000
processing page : 4
https://timesofindia.indiatimes.com/city/mumbai/4?cfmid=14000000
processing page : 5
https://timesofindia.indiatimes.com/city/mumbai/5?cfmid=14000000
processing page : 2
https://timesofindia.indiatimes.com/city/mumbai/2?cfmid=11000000
processing page : 3
https://timesofindia.indiatimes.com/city/mumbai/3?cfmid=11000000
processing page : 4
https://timesofindia.indiatimes.com/city/mumbai/4?cfmid=11000000
processing page : 5
https://timesofindia.indiatimes.com/city/mumbai/5

In [ ]:
print(len(news))

2942


In [ ]:
final_data = []
for i in range(len(news) // 2):
  title, link, city, topic = news[i]
  try:
      content = requests.get(link)                        
  except Exception as e:                                
      continue   
  new_soup = BeautifulSoup(content.text,'html.parser')
  data = new_soup.find('div', attrs={'class': "byline"})
  try:
    data = data.text.split("/")
  except: 
    continue
  # print(data)
  date = ''
  try:
    date = data[-1].replace('Updated:', "").strip("").strip()
  except:
    date = data[-1].strip("").strip()
  # print(date)
  author = data[0].strip()
  try:
    data = new_soup.find('section', attrs={'class': "undefined"}).next_siblings
  except:
    continue
  content = ''
  for i in data:
    content = i.text.split(":")[-1]
    # print(topic)
    break
  if len(title) == 0 or len(link) == 0 or len(city) == 0 or len(date) == 0 or len(content) == 0:
    continue
  final_data.append([title, link, city, date, author, content, topic])


In [ ]:
print(final_data[2])

["\nScooterist who was groped seeks Mumbai police, minister's help\n", 'https://timesofindia.indiatimes.com/city/mumbai/scooterist-who-was-groped-seeks-mumbai-police-ministers-help/articleshow/78420726.cms', 'mumbai', 'Oct 1, 2020, 11:02 IST', 'TNN', " A student on a scooter with her sister was groped at a busy street in Kurla by a man riding triple seat on a motorcycle. The sisters chased the bike, confronted the man and later uploaded a video clip of the confrontation on Twitter, tagging minister Aaditya Thackeray and the police commissioner. The student tweeted that one of them was singing while their bike was behind, and while moving past, the man in the middle touched her inappropriately on her back and they sped ahead near Phoenix Marketcity around 6.10pm on September 27.. The sisters chased the men through traffic, yelling at them to halt. Some bystanders gathered and when the women narrated what happened, the men were forced to get off the bike. An officer of Vinoba Bhave polic

In [ ]:
data_news = pd.DataFrame(index=range(len(final_data)), columns=['Title', 'Link', 'City', 'Date', 'Author' ,'Content', 'Topic'])
title = []
link = []
city = []
date = []
content = []
author = []
topic = []

for i in final_data:
  # print(i)
  # break
  title.append(i[0].strip("\n"))
  link.append(i[1])
  city.append(i[2])
  date.append(i[3])
  author.append(i[4])
  content.append(i[5])
  topic.append(i[6])

data_news['Title'] = title
data_news['Link'] = link
data_news['City'] = city
data_news['Date'] = date
data_news['Author'] = author
data_news['Content'] = content
data_news['Topic'] = topic


In [ ]:
data_news.to_csv("Data_News.csv")
print(data_news.head())

                                               Title  ...       Topic
0  Three men rob hotel on Mumbai-Ahmedabad nation...  ...  crime_news
1             Man arrested with fake notes in Mumbai  ...  crime_news
2  Scooterist who was groped seeks Mumbai police,...  ...  crime_news
3  Illegal Remdesivir sale: Pre-arrest bail petit...  ...  crime_news
4  Mumbai: Kidnapping accounts for 55% of crimes ...  ...  crime_news

[5 rows x 7 columns]
